# PydanticAI Agent
pydantic-ai 패키지를 사용한 agent 구성

[구성]
* level1 agent (A): text 분석 에이전트
    * level2 agent1 (B): sentiment-analysis agent
    * level2 agent2 (C): emotion-prediction agent


In [1]:
from enum import Enum
from typing import List, Optional
import yaml

from jinja2 import Template, StrictUndefined
from jinja2.exceptions import TemplateError

from pydantic import BaseModel, Extra, Field

from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel, OpenAIResponsesModelSettings
from pydantic_ai.providers.openai import OpenAIProvider

from pydantic_ai.models import ModelRequestParameters
from pydantic_ai.messages import ModelMessage
from pydantic_ai.settings import ModelSettings

from config import settings

# Model Init

OpenAIModel
* https://ai.pydantic.dev/models/#environment-variable
* https://github.com/pydantic/pydantic-ai/blob/1def7df728a5e757ceb16797a8b608e00da70982/pydantic_ai_slim/pydantic_ai/models/openai.py#L146

ModelRequestParameters
* Configuration for an agent's request to a model, specifically related to tools and output handling.


ModelMessage

ModelSettings

In [2]:
# Inint LLM Provider
print(settings.llm_model)
provider = OpenAIProvider(
    base_url=settings.llm_base_url,
    api_key=settings.llm_api_key
)
model = OpenAIModel(
    model_name=settings.llm_model,
    provider=provider
)

gpt-4.1-nano


In [4]:
with open('prompts/2_pydanticai.yaml', 'r') as file:
    prompts = yaml.load(file, Loader=yaml.FullLoader)
print(prompts.keys())

dict_keys(['agent_a', 'agent_b', 'agent_c'])


# 1. Init Agents

In [5]:
# Agent A (level1)
agent_a_system_message = prompts["agent_a"]["system"]
agent_a_user_template = Template(
    prompts["agent_a"]["user"],
    undefined=StrictUndefined
)

AgentA = Agent(
    model=model,
    name="text_analyzer",
    system_prompt=agent_a_system_message
)

In [6]:
# Agent B (level2) - sentiment analysis
class Polarity(str, Enum):
    positive = "positive"
    neutral = "neutral"
    negative = "negative"
    
class SentimentAnalysisResult(BaseModel):
    sentiment: Polarity
    reason: str
    
agent_b_user_template = Template(
    prompts["agent_b"]["user"],
    undefined=StrictUndefined
)
AgentB = Agent(
    model=model,
    name="sentiment_analyzer",
    system_prompt=prompts["agent_b"]["system"],
    output_type=SentimentAnalysisResult,
)

@AgentA.tool
async def analyze_sentiment(
    ctx: RunContext[None],
    text: str,
)->Optional[Polarity]:
    print("[TOOL] analyze_sentiment")
    result = await AgentB.run(
        agent_b_user_template.render(text=text)
    )
    output = result.output
    print(output)
    return output.sentiment.value

In [7]:
await analyze_sentiment(ctx=None, text="I am happy")

[TOOL] analyze_sentiment
sentiment=<Polarity.positive: 'positive'> reason='The text expresses happiness and a positive emotional state.'


'positive'

In [8]:
# Agent C (level2) - emotion analysis
class EmotionLevel(str, Enum):
    low = "low"
    medium = "medium"
    high = "high"
    
class Emotion(BaseModel):
    joy: EmotionLevel
    anger: EmotionLevel
    
class EmotionAnalysisResult(BaseModel):
    emotion: Emotion
    
agent_c_user_template = Template(
    prompts["agent_c"]["user"],
    undefined=StrictUndefined
)
AgentC = Agent(
    model=model,
    name="emotion_analyzer",
    system_prompt=prompts["agent_c"]["system"],
    output_type=EmotionAnalysisResult,
)

@AgentA.tool
async def analyze_emotion(
    ctx: RunContext[None],
    text: str,
)->Optional[Emotion]:
    print("[TOOL] analyze_emotion")
    result = await AgentC.run(
        agent_b_user_template.render(text=text)
    )
    output = result.output
    print(output)
    return output.emotion

In [9]:
await analyze_emotion(ctx=None, text="I am happy")

[TOOL] analyze_emotion
emotion=Emotion(joy=<EmotionLevel.high: 'high'>, anger=<EmotionLevel.low: 'low'>)


Emotion(joy=<EmotionLevel.high: 'high'>, anger=<EmotionLevel.low: 'low'>)

# 2. Agent Run

In [10]:
text = "I was having a good day but I got hit by a car"
result = await AgentA.run(
    agent_a_user_template.render(text=text)
)

[TOOL] analyze_sentiment
[TOOL] analyze_emotion
sentiment=<Polarity.negative: 'negative'> reason='The person was having a good day but experienced a negative event (getting hit by a car), which indicates a negative sentiment.'
emotion=Emotion(joy=<EmotionLevel.low: 'low'>, anger=<EmotionLevel.medium: 'medium'>)


In [11]:
result

AgentRunResult(output='The overall sentiment of the text is negative. Emotionally, the person initially felt joy or positivity, but the experience of being hit by a car has introduced feelings of anger and likely distress. This suggests a shift from a good mood to negative emotions due to an unfortunate incident.')

In [12]:
import pprint
pprint.pprint(result.output)

('The overall sentiment of the text is negative. Emotionally, the person '
 'initially felt joy or positivity, but the experience of being hit by a car '
 'has introduced feelings of anger and likely distress. This suggests a shift '
 'from a good mood to negative emotions due to an unfortunate incident.')
